### 학습목표
- 네이버 블로그 데이터 수집하기
- 네이버 카페 데이터를 수집했던 경험을 통하여 동일한 프로세스로 블로그 글 수집하기~
- 수집 후 워드클라우드 생성

##### 실습 순서
1. "음식물 처리기 사용 후기" 검색결과를 블로그 링크 분석
2. 키워드 검색, 기간설정이 가능한 url 을 생성 (6개월)
3. 드라이버를 통하여 페이지 요청
4. 스크롤 내리기 10번
5. 블로그 주소 수집 (href_list)
6. 블로그 접근하여 본문 수집 (iframe)
7. 코트 통합하여 모든 블로그글이 수집
8. 텍스트 데이터 txt 파일로 저장
9. 텍스트 전처리
10. 워드클라우드 생성

In [1]:
# 라이브러리 불러오기
import selenium.webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import pandas as pd
import re # 정규표현식

# 문자열 전처리 함수 -> 숫자, 문자, (?.!,¿) 제외하고 공백으로 대체
def preprocess_sentence_kr(w):
  w = w.strip()
  w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) 
  w = w.strip() 
  return w

1. "음식물 처리기 사용 후기" 검색결과 블로그 링크 분석하기
   - 키워드 확인
   - 기간확인 -> 6m, 3m,1y  
url 내에 포매팅

- https://search.naver.com/search.naver?ssc=tab.blog.all&query=%EC%9D%8C%EC%8B%9D%EB%AC%BC%20%EC%B2%98%EB%A6%AC%EA%B8%B0%20%EC%82%AC%EC%9A%A9%20%ED%9B%84%EA%B8%B0&sm=tab_opt&nso=so%3Ar%2Cp%3A3m
- https://search.naver.com/search.naver?ssc=tab.blog.all&query=%EC%9D%8C%EC%8B%9D%EB%AC%BC%20%EC%B2%98%EB%A6%AC%EA%B8%B0%20%EC%82%AC%EC%9A%A9%20%ED%9B%84%EA%B8%B0&sm=tab_opt&nso=so%3Ar%2Cp%3A6m
- https://search.naver.com/search.naver?ssc=tab.blog.all&query=%EC%9D%8C%EC%8B%9D%EB%AC%BC%20%EC%B2%98%EB%A6%AC%EA%B8%B0%20%EC%82%AC%EC%9A%A9%20%ED%9B%84%EA%B8%B0&sm=tab_opt&nso=so%3Ar%2Cp%3A1y

2. 키워드 검색, 기간설정이 가능한 url 을 생성 (6개월)

In [3]:
from urllib.parse import quote
keyword = quote('음식물 처리기 사용 후기')
term = '6m' #직접기간설정도 가능함 from20250101to20251231
url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&query={keyword}&sm=tab_opt&nso=so%3Ar%2Cp%3A{term}"
url

'https://search.naver.com/search.naver?ssc=tab.blog.all&query=%EC%9D%8C%EC%8B%9D%EB%AC%BC%20%EC%B2%98%EB%A6%AC%EA%B8%B0%20%EC%82%AC%EC%9A%A9%20%ED%9B%84%EA%B8%B0&sm=tab_opt&nso=so%3Ar%2Cp%3A6m'

3. 드라이버를 통하여 페이지 요청

In [5]:
driver = wb.Chrome()
driver.get(url)

4. 스크롤 내리기 10번

In [6]:
for i in range(10):
    body = driver.find_element(By.TAG_NAME,'body')
    body.send_keys(Keys.END)
    time.sleep(1)

5. 블로그 주소 수집 (href_list)

In [10]:
aTag = driver.find_elements(By.CSS_SELECTOR, 'a.fender-ui_228e3bd1.yEnIcjNmMJN3_9CrNfSS')
len(aTag)

330

In [11]:
href_list = []
for h in aTag:
    href = h.get_attribute('href')
    href_list.append(href)

In [12]:
href_list

['https://blog.naver.com/fbslsk123/224146664438',
 'https://blog.naver.com/modellien/224150332221',
 'https://blog.naver.com/wse1233/224106903217',
 'https://blog.naver.com/forever_cape/224144131131',
 'https://blog.naver.com/estherdanbi/224029989386',
 'https://blog.naver.com/jin03260326/224072786863',
 'https://blog.naver.com/iamkyungwon/224140944004',
 'https://blog.naver.com/cinewriter/224159964769',
 'https://blog.naver.com/sarah6612/224106012370',
 'https://blog.naver.com/ryeosan/223974783091',
 'https://blog.naver.com/for_sm/224137040456',
 'https://blog.naver.com/cutemiru/224135466124',
 'https://blog.naver.com/journey0703/224083824765',
 'https://blog.naver.com/onlyeunja/224090445742',
 'https://blog.naver.com/foryou1035/224153385544',
 'https://blog.naver.com/wjsalstmd00/223996682228',
 'https://blog.naver.com/forever_cape/224146810713',
 'https://blog.naver.com/sugidanji/224041368270',
 'https://blog.naver.com/smallview/224135413316',
 'https://blog.naver.com/jasonmoondistri

6. 블로그 접근하여 본문 수집 (iframe)

In [13]:
# 첫번째 주소의 본문 수집
driver.get(href_list[0])

In [14]:
driver.find_element(By.CLASS_NAME,'se-main-container')
# iframe

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".se-main-container"}
  (Session info: chrome=144.0.7559.60); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff730a25715
	0x7ff730a25770
	0x7ff7307d108d
	0x7ff73082c3be
	0x7ff73082c6cc
	0x7ff73087d2f7
	0x7ff730879e45
	0x7ff73081cc28
	0x7ff73081db33
	0x7ff730d0e9f0
	0x7ff730d08ded
	0x7ff730d2969a
	0x7ff730a411e5
	0x7ff730a499cc
	0x7ff730a2ebf4
	0x7ff730a2eda6
	0x7ff730a14e87
	0x7ff9c7fc7374
	0x7ff9c927cc91


In [15]:
# iframe 태그 접근
driver.switch_to.frame('mainFrame')

In [17]:
driver.find_element(By.CLASS_NAME,'se-main-container').text

'오늘은 음식물처리기 일반쓰레기 사용 방법 및 처리 후기를 남겨 볼게요.\n저는 어려서부터 비위 약한 걸 못 봤어요!\n그런데 결혼하고 나니까, 이것 때문에 골병 썩게 됐는데요. 남편이 올 때까지 치우지 않으면 냄새가 올라오더라고요. 집에는 악취가 쌓이고요.\n그래서 최후의 수단으로, 음식물처리기를 사야겠다고 마음 먹었습니다. 흔쾌히 오케이 해주어서, 바로 좋은 제품을 찾아보기로 했죠.\n음식물처리기 일반쓰레기 제품은 에어메이드 더 브릭스 AFC-451이에요.\n직접 사용해보면서 느꼈던 점이 많았는데요. 막상 쓰기 시작하니까, 너무 편해서 사길 잘했다는 생각이 저절로 들 정도랍니다.\n그러면 바로 사용 방법과 처리 후기를 남겨볼게요!\n우선 제가 궁금했던 건 “분쇄가 된 음식물은 일반쓰레기에 버려도 될까?”였어요.\n음식물처리기 일반쓰레기 공용 사용은 불가능이었습니다.\n즉, 음식물쓰레기는 일반쓰레기로 버리면 안 된다는 이야기가 많아요. 건조, 분쇄가 되었다고 하더라고, 폐기물의 종류가 바뀌는 게 아니라는 의견 때문이라고 합니다!\n음식물처리기 일반쓰레기 차이는 음쓰에 유기물 성분이 그대로 남아있기 때문이었어요.\n그렇기 때문에, 향후 사료나 퇴비로 사용될 수 있다는 점을 고려해! 일반 쓰레기로 버리면 안 된다는 말에 힘을 실어주었죠.\n그래서 저희는 열풍이 되는 상품을 구매했지만, 여전히 음쓰 봉지에 버리고 있습니다. 좋은 점은 냄새가 안 난다는 점!\n그래서 서로 다르게 버려야 한다는 걸 인지하고, 제품을 알아 봤습니다!\n종류는 미생물, 발효, 건조, 열풍건조 등이 있었어요. 저는 열풍으로 건조해주고 분쇄해주는 방식이 더 마음에 들었는데요. 찾아 보니까 위 아래서 열을 가해주기 때문에, 더 확실하게 건조가 되면서 깨끗하게 분쇄가 된다더라고요.\n음식물처리기 사용 방법은 음식물을 모아, 한 데에 넣어 놓고 ‘시작’ 버튼만 누르면 되는 거였어요.\n추운 날 밖에 나가지 않고도, 빠르게 처리할 수 있음에 감사했는데요. 빠르게 처리하는 것도 좋은데, 냄새가 나지

7. 코트 통합하여 모든 블로그글이 수집
8. 텍스트 데이터 txt 파일로 저장
9. 텍스트 전처리

In [19]:
driver = wb.Chrome()
f = open('./data/네이버 블로그 수집.txt','w')
for i in tqdm(range(len(href_list))):
    driver.get(href_list[i])
    time.sleep(2)

    driver.switch_to.frame('mainFrame')
    result = driver.find_element(By.CLASS_NAME,'se-main-container')
    content = preprocess_sentence_kr(result.text)
    f.write(content)
    time.sleep(1)

f.close()    
driver.quit()

100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [20:44<00:00,  3.77s/it]


10. 워드클라우드 생성

In [20]:
f = open('./data/네이버 블로그 수집.txt','r')
text = f.readline()
f.close()

In [ ]:
from kiwipiepy import Kiwi
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
from kiwipiepy import Kiwi
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

kiwi = Kiwi()
token = kiwi.tokenize(text)
nn_list = []
for t in token:
    if t.tag in ['NNG','NNP']:
        nn_list.append(t.form)
counter = Counter(nn_list)
top_100 = counter.most_common(100)

wc = WordCloud(
    font_path='C:/Windows/Fonts/malgunbd.ttf',
    background_color='white' , 
    # mask = mask_img,
    # colormap = 'winter'
).generate_from_frequencies(dict(nn_list2))
plt.imshow(wc)
plt.axis('off')
plt.show()